This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,   0.        ,
        47.20790294,   0.        ]), array([236.48363759, 169.98335839, 144.91973485, ...,  66.92658557,
        55.51053098,   7.33894856]), OrderedDict([('R2', 0.7596954611738436), ('root_mean_squared_error', 47.402609232310056), ('mean_absolute_error', 32.21340207853417), ('rmse_over_stdev', 0.49418665079119495)]), None)
test_quad = (array([2.22927321e+02, 2.14185752e+02, 1.82129033e+02, 3.96883335e+02,
       3.88395179e+02, 2.44558934e+02, 7.98843255e+01, 1.21202059e+02,
       1.43144270e+02, 1.10503751e+02, 1.29676557e+02, 1.06566685e+02,
       2.14252916e+02, 3.04216098e+02, 1.78590946e+02, 1.11989369e+02,
       1.47491884e+02, 9.77537153e+01, 2.33371776e+02, 1.88725323e+02,
       1.44367703e+02, 3.21795003e+02, 1.96095876e+02, 1.25081235e+02,
       1.37316410e+02, 1.72040539e+02, 1.85728826e+02, 1.14367926e+02,
       1.37488659e+02, 1.27740217e+02, 1.56941241e+02, 1.58367410e+02,
       1.21204438e+02, 4.57531836e+01, 2.16105345e+01, 2.89476692e+01,
       9.78662706e+01, 2.93743989e+02, 8.60496631e+01, 9.70462162e+01,
       1.11442294e+02, 6.36339325e+02, 1.56229730e+02, 2.36664682e+02,
       9.25437335e+01, 2.01817565e+02, 1.34180693e+02, 1.02036436e+02,
       1.82740912e+02, 2.38334229e+02, 7.34978297e+01, 7.14456318e+01,
       6.43729766e+02, 7.66704600e+00, 9.04068336e+01, 6.92869614e+01,
       6.36890317e+01, 1.14525327e+02, 5.00993000e+01, 0.00000000e+00,
       1.41664751e+02, 7.38023139e+01, 6.31512971e+01, 1.01797990e+02,
       0.00000000e+00, 1.05881855e+01, 2.26412639e+01, 2.99180823e+01,
       9.68586984e+01, 1.12003551e+02, 1.27239967e+02, 2.12302521e+02,
       1.51797441e+02, 2.89712978e+02, 2.51771744e+02, 1.50472708e+02,
       4.31928159e+01, 3.31384554e+01, 4.52159618e+01, 6.21679659e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.92094295e+00,
       1.99014786e+01, 0.00000000e+00, 1.69769380e+02, 1.39370004e+02,
       7.53897668e+01, 1.35168013e+02, 0.00000000e+00, 4.06187327e+01,
       3.81461657e+01, 6.05219124e+01, 5.67517336e+01, 4.89943291e+01,
       0.00000000e+00, 3.13433527e+01, 1.81910775e+02, 1.50508186e+02,
       1.08979858e+02, 8.64067635e+01, 1.14247314e+02, 1.15200294e+02,
       1.02514386e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.19314875e+01, 4.93632300e+01, 1.62043592e+01,
       0.00000000e+00, 1.48100516e+01, 5.26296357e+01, 6.06802766e+01,
       7.37896699e+00, 0.00000000e+00, 2.57222520e+01, 2.21842737e+01,
       9.55205228e+01, 3.73488338e+01, 0.00000000e+00, 6.12137026e+01,
       3.51232216e+01, 0.00000000e+00, 1.17714484e+02, 5.35428595e+01,
       1.12573845e+02, 2.68325976e+01, 5.86508558e+01, 1.29323672e+02,
       1.75579596e+02, 0.00000000e+00, 2.02289181e-01, 2.24784003e+01,
       1.37126059e+02, 7.31812963e+01, 1.25292208e+02, 0.00000000e+00,
       9.36717096e+01, 0.00000000e+00, 8.69178900e+00, 2.63839575e+02,
       2.76340269e+02, 8.39171957e+01, 2.51360287e+02, 4.73693459e+01,
       2.81102443e+02, 1.40538550e+02, 2.29064683e+02, 3.67624432e+02,
       1.99684569e+02, 4.96917625e+01, 2.53031123e+01, 0.00000000e+00,
       3.29759565e+01, 1.38924377e+01, 1.05110827e+01, 0.00000000e+00,
       0.00000000e+00, 2.15174731e+01, 5.12388154e+01, 0.00000000e+00,
       0.00000000e+00, 4.81001678e+01, 5.40734041e+01, 1.32773863e+02,
       2.04099993e+02, 7.67577252e+01, 3.81230146e+01, 3.74738040e+01,
       1.57158326e+02, 5.07080997e+01, 0.00000000e+00, 0.00000000e+00,
       3.42510607e+01, 3.13514481e+01, 1.52429429e+02, 8.15909270e+01,
       9.43019059e+01, 1.41373726e+02, 4.00673869e+01, 2.50852147e+01,
       2.67496464e+01, 2.85835223e+01, 6.33172566e+01, 3.63567935e+01,
       1.76733958e+01, 2.57130076e+01, 2.75959946e+01, 5.17559553e+01,
       8.22953902e+01, 7.16777317e+01, 1.01745118e+01, 4.94252903e+01,
       1.05442368e+02, 1.77187703e+01, 0.00000000e+00, 5.43550612e+00,
       1.78137720e+02, 1.89735033e+01, 2.11232453e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.61974495e+01,
       0.00000000e+00, 0.00000000e+00, 3.56929084e+01, 7.37535702e+01,
       1.42333954e+02, 2.44348237e+02, 1.55663046e+02, 1.57242981e+02,
       2.60129737e+02, 2.17252824e+02, 1.79654132e+02, 1.83828681e+02,
       1.65118221e+02, 2.75656178e+02, 2.87661839e+02, 3.67138367e+02,
       8.15746890e+01, 1.30390352e+02, 1.03756755e+02, 1.00207954e+02,
       2.59886084e+02, 1.96409134e+02, 1.44708713e+02, 1.22677657e+02,
       1.44006558e+02, 1.33954726e+02, 1.06220359e+02, 1.22431670e+02,
       1.21428977e+02, 2.31128112e+02, 1.18248560e+02, 1.77752168e+02,
       3.19116631e+02, 1.69573331e+02, 2.22246511e+02, 3.23822637e+02,
       2.04046571e+02, 1.30558998e+02, 2.47684975e+02, 2.52161354e+02,
       2.96095913e+02, 9.69267962e+01, 1.54641811e+02, 1.98592674e+02,
       3.54737752e+02, 3.53349574e+02, 3.38955480e+02, 3.30939913e+02,
       1.79999332e+02, 1.44889042e+02, 3.50052571e+02, 3.49605405e+02,
       2.18137285e+02, 2.23307827e+02, 7.10448978e+01, 1.44154498e+02,
       1.34599602e+02, 1.50042028e+02, 2.23624761e+02, 1.65109730e+02,
       6.45557199e+01, 6.15763632e+01, 1.50801863e+02, 1.26818170e+02,
       4.52932141e+01, 1.17792277e+02, 2.33383666e+02, 2.83042429e+02,
       1.32211588e+02, 2.51703483e+02, 1.61637650e+02, 9.69622803e+01,
       5.74165434e+01, 1.10065846e+02, 1.09913097e+02, 1.31798616e+02,
       1.22185191e+02, 1.02217058e+02, 2.66189733e+01, 1.74303139e+01,
       3.51009606e+01, 9.31208466e+01, 7.34595746e+01, 1.27664645e+02,
       4.18854297e+01, 3.05328625e+01, 1.82785945e+01, 3.31838908e+01,
       7.58528445e+01, 1.73730028e+02, 2.05075217e+02, 1.62590223e+02,
       1.76433607e+02, 1.47336842e+02, 1.98013501e+02, 1.67324805e+02,
       1.70292754e+02, 1.87207178e+02, 2.36861914e+02, 1.65127413e+02,
       1.33312148e+02, 1.11384136e+02, 1.42933388e+02, 1.79710356e+02,
       1.84787732e+02, 1.37406686e+02, 0.00000000e+00, 1.24542111e+02,
       1.59403546e+02, 8.35686242e+01, 1.37479653e+02, 1.09488956e+02,
       9.60360384e+01, 8.66997511e+01, 1.17249168e+02, 9.79185800e+01,
       1.83151200e+02, 1.71490363e+02, 1.15594393e+02, 1.35955483e+02,
       1.45310469e+02, 1.40364781e+02, 1.63497124e+02, 9.12669747e+01,
       8.76081324e+01, 9.37481720e+01, 7.98648319e+01, 6.69631063e+01,
       3.63667394e+01, 2.00619416e+01, 3.86328698e+01, 3.97988019e+01,
       5.91429461e+01, 7.14796192e+01, 7.26752642e+01, 6.16998797e+01,
       9.88464407e+01, 3.41670362e+02, 1.43739984e+02, 8.70884029e+01,
       9.43179777e+01, 1.82417160e+02, 1.79369514e+02, 3.81360533e+01,
       1.10110223e+02, 0.00000000e+00, 6.62624423e+01, 0.00000000e+00,
       1.12174952e+02, 7.18885960e+01, 1.24759381e+02, 1.98105436e+02,
       2.15058665e+01, 4.75557533e+00, 1.13483054e+02, 1.36252990e+02,
       6.23247673e+01, 5.63388158e+01, 5.63192894e+01, 5.92192098e+01,
       6.24481656e+01, 1.21974896e+02, 0.00000000e+00, 4.87758689e+01,
       4.34643306e+01, 1.24011708e+01, 2.35812529e+01, 2.71196539e+01,
       3.40153297e+01, 4.92018258e+01, 2.07757183e+02, 7.56574945e+01,
       8.29015442e+01, 6.98177552e+01, 6.74148765e+01, 7.04541654e+01,
       6.55202594e+01, 1.21693503e+02, 9.11059876e+01, 3.91108879e+01,
       4.54995709e+01, 4.50377785e+01, 4.82400402e+01, 3.39806022e+01,
       4.02393064e+01, 1.66347669e+02, 1.23233184e+02, 1.10664774e+02,
       1.05624926e+02, 1.42323884e+02, 3.15042600e+01, 3.74221989e+01,
       2.92307342e+01, 2.07374692e+01, 5.90626223e+01, 1.60395108e+02,
       4.26966290e+01, 1.20061041e+02, 7.00432397e+01, 3.53717988e+01,
       5.24254612e+01, 7.76829500e+01, 1.34709235e+01, 6.57797960e+01,
       1.21777150e+02, 2.19120080e+01, 7.17549475e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 8.66887169e+00, 0.00000000e+00,
       2.38090660e+01]), array([ 2.05435371e+02,  1.68588256e+02,  1.70104105e+02,  2.67162467e+02,
        2.81622968e+02,  2.18397900e+02,  1.03053514e+02,  1.72764640e+02,
        1.16533832e+02,  1.21401439e+02,  8.67592637e+01,  9.22638256e+01,
        1.52548624e+02,  2.86865135e+02,  1.96733047e+02,  1.76668157e+02,
        1.17139826e+02,  9.84491588e+01,  2.32899624e+02,  1.69552534e+02,
        2.31937888e+02,  2.91922440e+02,  1.65442839e+02,  1.51356205e+02,
        1.53438827e+02,  1.98011096e+02,  1.91946267e+02,  1.51024681e+02,
        1.12957409e+02,  1.24900849e+02,  1.48144627e+02,  1.59775323e+02,
        1.30251639e+02,  6.99071307e+01,  3.75690526e+01,  5.07788965e+01,
        1.30662100e+02,  1.66647380e+02,  1.04557859e+02,  1.05975481e+02,
        1.18663890e+02,  5.82273429e+02,  1.70230724e+02,  2.26147366e+02,
        1.17667838e+02,  1.29451449e+02,  1.44766870e+02,  1.55199004e+02,
        1.77885773e+02,  1.71607797e+02,  6.95786543e+01,  7.31104904e+01,
        2.97897345e+02,  2.08228537e+01,  2.20981991e+02,  1.39970243e+02,
        1.96315339e+02,  1.99738624e+02,  3.97542671e+01,  2.05031922e+01,
        1.30559196e+02,  1.13120740e+02,  1.18084596e+02,  1.47585264e+02,
        3.45333504e+01,  4.18534742e+01,  5.10659722e+01, -8.67932125e-01,
        5.14057913e+01,  1.04841881e+02,  1.53435247e+02,  1.39561232e+02,
        1.47310979e+02,  1.69987991e+02,  1.65995511e+02,  1.61674807e+02,
        4.49857224e+01,  7.23596873e+01,  8.51948945e+01,  6.61610506e+01,
       -2.77959961e+00,  3.33767499e+01,  3.80570918e+01,  3.25093791e+01,
        5.48197201e+01, -4.56583036e+00,  2.48382700e+01,  2.62542249e+01,
        5.03770956e+01,  7.00501138e+01,  2.50055415e+01,  4.03814067e+01,
        2.61164442e+01,  4.08173149e+01,  1.45815523e+02,  1.33187388e+02,
        8.38051062e+01,  1.34099814e+02,  3.41173361e+01,  8.54660146e+01,
        6.70618023e+01,  4.80264889e+01,  3.91127243e+01,  4.32445129e+01,
        1.94869246e+01,  5.71949740e+01,  1.10593381e+02,  1.12916908e+02,
        9.85794651e+01,  8.84498310e+01,  9.01889685e+01,  9.29150486e+01,
        8.09485380e+01,  1.83126661e+01,  2.13944237e+01,  2.18272434e+01,
        1.99970705e+01,  4.48132454e+01,  7.20240973e+01,  8.29043259e+01,
        2.11255651e+01,  2.86918319e+01,  3.80521538e+01,  5.22835820e+01,
        2.61240445e+01,  2.75083692e+01,  2.47803790e+01,  2.19818999e+01,
        3.26817933e+01,  3.19638455e+01,  2.34702455e+01,  4.79991234e+01,
        4.75214812e+01,  3.71946986e+00,  5.60603631e+01,  7.15939820e+01,
        1.18676333e+02,  3.45067667e+01,  1.69740302e+01,  1.84724278e+02,
        2.36105620e+02,  3.37261893e+01,  2.05758425e+01,  3.74327242e+01,
        6.58655606e+01,  1.19398373e-04,  2.43957773e+02,  7.70389422e+01,
        1.19876981e+02,  3.92610863e+01,  3.67416764e+01,  2.75970271e+02,
        2.60142524e+02,  9.83335297e+01,  4.88321960e+01,  7.89413664e+01,
        2.01251581e+02,  9.50178194e+01,  1.06480484e+02,  2.13246447e+02,
        7.52983586e+01,  7.05393971e+01,  6.57020880e+01,  1.83433584e+01,
        6.15732522e+01,  3.67717262e+01,  1.91062285e+01,  1.41757402e+01,
        5.82289812e+00,  3.21238758e+01,  5.67103453e+01,  1.45339008e+01,
        2.10475361e+01,  4.21933346e+01,  2.89203246e+01,  1.12369736e+02,
        1.55147974e+02,  8.43680318e+01,  6.03426389e+01,  9.16526453e+01,
        6.91794791e+01,  5.92415812e+01, -2.14204698e+00,  4.20468750e-01,
        4.41026169e+01,  4.01961951e+01,  1.60416226e+02,  1.00705752e+02,
        8.67013994e+01,  8.29982145e+01,  3.50595968e+01,  3.27005424e+01,
        3.28668758e+01,  3.31471495e+01,  5.78804575e+01,  7.30288788e+01,
        2.47844734e+01,  4.73830455e+01,  5.17257108e+01,  4.88515106e+01,
        6.32251082e+01,  7.44002106e+01,  3.55411206e+01,  8.20599141e+00,
        3.18893902e+01,  3.70306429e+01,  3.49102918e+01,  3.23123765e+01,
        4.84033449e+01,  2.52473191e+01,  2.13318728e+01,  4.50711016e+01,
        2.58604664e+01,  1.14761255e+01,  1.16477893e+01,  1.38095346e+01,
        1.45931274e+01,  1.33917529e+01,  1.48827513e+01,  2.89103617e+01,
        1.76316547e+02,  2.77344700e+02,  1.43752687e+02,  1.32177310e+02,
        2.81424768e+02,  3.05281365e+02,  1.55111431e+02,  1.31235726e+02,
        1.25091048e+02,  1.70996941e+02,  2.66399095e+02,  3.01039821e+02,
        1.09595489e+02,  1.59126816e+02,  9.38359540e+01,  7.87823319e+01,
        1.70679433e+02,  1.36371218e+02,  1.32194501e+02,  8.41612078e+01,
        1.02746145e+02,  1.29947007e+02,  7.14494242e+01,  1.18594772e+02,
        1.21413454e+02,  1.49181279e+02,  1.39989982e+02,  1.48658473e+02,
        1.91366862e+02,  1.37759017e+02,  2.69463428e+02,  3.06056586e+02,
        2.17316621e+02,  1.63846519e+02,  2.41736858e+02,  1.67277915e+02,
        2.80130305e+02,  9.81280074e+01,  1.29896129e+02,  1.34844648e+02,
        3.09352692e+02,  2.52359328e+02,  2.36754908e+02,  2.88421635e+02,
        1.71407668e+02,  1.55746351e+02,  2.37599025e+02,  3.04294658e+02,
        1.65304581e+02,  1.87479299e+02,  8.48509054e+01,  1.71425449e+02,
        1.61298793e+02,  1.48668862e+02,  2.69861045e+02,  1.31489046e+02,
        8.07699083e+01,  6.91655725e+01,  8.44519889e+01,  1.19503262e+02,
        7.61181694e+01,  1.33357123e+02,  1.39541001e+02,  3.02629223e+02,
        1.65629831e+02,  2.97457241e+02,  1.39846183e+02,  1.27251936e+02,
        1.12912187e+02,  1.42191898e+02,  1.31311134e+02,  1.28514085e+02,
        1.37430305e+02,  1.28087995e+02,  7.03539248e+01,  3.90208352e+01,
        4.20025524e+01,  4.27501360e+01,  1.09380118e+02,  1.36031773e+02,
        5.58256500e+01,  7.20628355e+01,  3.82621773e+01,  3.90879176e+01,
        1.18846515e+02,  1.75703457e+02,  1.69305608e+02,  1.76322255e+02,
        1.81979201e+02,  1.80579908e+02,  1.82188805e+02,  1.16310489e+02,
        2.46352016e+02,  1.85084120e+02,  1.75624073e+02,  1.80369608e+02,
        1.30365125e+02,  1.16374775e+02,  1.33894211e+02,  1.76485888e+02,
        1.81077100e+02,  1.17769310e+02,  7.65152962e+01,  2.28796049e+02,
        1.35936569e+02,  1.25414869e+02,  1.59982787e+02,  1.34293301e+02,
        1.06164474e+02,  1.06597764e+02,  1.23117200e+02,  1.14459506e+02,
        1.40673414e+02,  1.12430616e+02,  1.14207618e+02,  1.43940021e+02,
        2.05032497e+02,  1.13255056e+02,  1.44175713e+02,  1.87604457e+02,
        1.08912820e+02,  1.25983708e+02,  1.00892328e+02,  7.30853337e+01,
        3.53327361e+01,  4.05552686e+01,  6.09552119e+01,  8.34967280e+01,
        8.64955577e+01,  9.64827936e+01,  9.89719781e+01,  1.05613916e+02,
        1.08106687e+02,  1.56294910e+02,  1.14620722e+02,  7.15937194e+01,
        1.19329714e+02,  1.17416653e+02,  1.50473031e+02,  6.11421105e+01,
        1.53859560e+02,  7.17721085e+00,  1.10416759e+02,  6.75572014e+00,
        4.65371006e+01,  7.84295792e+01,  1.09898530e+02,  1.52492789e+02,
        3.86475298e+01,  2.24117790e+01,  7.60247654e+01,  9.68763971e+01,
        7.10471198e+01,  8.61870044e+01,  5.34937772e+01,  9.90038207e+01,
        5.19542043e+01,  5.84833110e+01,  7.02356832e+00,  5.16279930e+01,
        3.70047913e+01,  3.46591469e+01,  2.28314779e+01,  3.18683866e+01,
        4.63802911e+01,  6.02898244e+01,  1.46006859e+02,  4.64907791e+01,
        5.59698568e+01,  7.30860612e+01,  9.05484117e+01,  4.49044313e+01,
        6.18058972e+01,  1.26177422e+02,  5.84323945e+01,  6.62663378e+01,
        3.10332707e+01,  3.53081357e+01,  5.19477788e+01,  3.79556217e+01,
        3.06631697e+01,  1.97613326e+02,  1.44924544e+02,  8.12027525e+01,
        7.28398255e+01,  7.87339223e+01,  2.67193555e+01,  2.23643764e+01,
        1.40681753e+01,  5.71865439e+01,  3.96227792e+01,  8.91059438e+01,
        1.97091245e+01,  5.25086832e+01,  3.87337544e+01,  4.62103936e+01,
        5.38015821e+01,  4.95854063e+01,  5.08914973e+01,  2.86009216e+01,
        8.36035526e+01,  5.12178482e+01,  3.15954537e+01,  1.05105557e+01,
        3.03319099e+01,  2.11618346e+01,  4.91328211e+01,  2.41388011e-01,
        4.48409355e+01]), OrderedDict([('R2', 0.7603765692707201), ('root_mean_squared_error', 46.1415397335229), ('mean_absolute_error', 31.89018694071966), ('rmse_over_stdev', 0.48103961685965824)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_9'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
